## Check data if present in the list is same as that of dataset.json for all the data.

In [1]:
import json
import os

In [2]:
import tqdm

In [2]:
#load the txt file
data_txt_file = './dataset/dataset_list/PAOT_10_inner_val.txt'

In [11]:
#go to each msd folder and get their dataset json file
dic = {}
msd_folder = './10_Decathlon'
for fold in os.listdir(msd_folder):
    fold_path = os.path.join(msd_folder,fold)
    if os.path.isdir(fold_path) and fold[4:6] in ['03','06','10','07','08','09']:
        json_path = os.path.join(fold_path,'dataset.json')
        #print(json_path)
        if os.path.exists(json_path):
            dic[fold_path]=json_path

In [12]:
dic

{'./10_Decathlon/Task09_Spleen': './10_Decathlon/Task09_Spleen/dataset.json',
 './10_Decathlon/Task07_Pancreas': './10_Decathlon/Task07_Pancreas/dataset.json',
 './10_Decathlon/Task03_Liver': './10_Decathlon/Task03_Liver/dataset.json',
 './10_Decathlon/Task06_Lung': './10_Decathlon/Task06_Lung/dataset.json',
 './10_Decathlon/Task10_Colon': './10_Decathlon/Task10_Colon/dataset.json',
 './10_Decathlon/Task08_HepaticVessel': './10_Decathlon/Task08_HepaticVessel/dataset.json'}

In [24]:
#well we have the files. We have to remove the '.' file from each folder
#also need to rename task with Dataset.
nnunet_raw = '/blue/kgong/s.kapoor/language_guided_segmentation/CLIP-Driven-Universal-Model_MSD_only/nnUNet_raw'

In [25]:
old_name = 'Task'
new_name = 'Dataset0'
for fold in os.listdir(nnunet_raw):
    if old_name in fold:
        source_path = os.path.join(nnunet_raw,fold)
        dst_path = os.path.join(nnunet_raw,fold.replace(old_name,new_name))
        os.rename(source_path,dst_path)

In [26]:
new_name2 = 'Dataset'
#now remove the . files
for fold in tqdm.tqdm(os.listdir(nnunet_raw)):
    if new_name2 in fold:
        fold_path = os.path.join(nnunet_raw,fold)
        for subfold in os.listdir(fold_path):
            subfold_path = os.path.join(fold_path,subfold)
            if os.path.isdir(subfold_path):
                for file in os.listdir(subfold_path):
                    file_path = os.path.join(subfold_path,file)
                    if file.startswith('.') and os.path.isfile(file_path):
                        os.remove(file_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 10.45it/s]


In [27]:
#now add _0000 to eachfile name cause modality.
des_fold = 'imagesTr'
for fold in tqdm.tqdm(os.listdir(nnunet_raw)):
    if new_name2 in fold:
        fold_path = os.path.join(nnunet_raw,fold)
        for subfold in os.listdir(fold_path):
            if not subfold.startswith('.') and des_fold in subfold:
                subfold_path = os.path.join(fold_path,subfold)
                for filename in os.listdir(subfold_path):
                    new_filename = filename[:-7]+'_0000.nii.gz'
                    filename_path = os.path.join(subfold_path,filename)
                    new_filename_path = os.path.join(subfold_path,new_filename)
                    os.rename(filename_path,new_filename_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 17.36it/s]


In [28]:
!pwd

/blue/kgong/s.kapoor/language_guided_segmentation/CLIP-Driven-Universal-Model_MSD_only


In [29]:
from generate_dataset_json import generate_dataset_json

In [33]:
#first replace the old file with dataset.json as dataset_nnunetv1.json
for fold in tqdm.tqdm(os.listdir(nnunet_raw)):
    if new_name2 in fold:
        fold_path = os.path.join(nnunet_raw,fold)
        json_file_path = os.path.join(fold_path,'dataset.json')
        new_json_file_path = os.path.join(fold_path,'dataset_nnunetv1.json')
        os.rename(json_file_path,new_json_file_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 2048.83it/s]


In [36]:
#now need to create dataset.json file for each dataset. Moreover it should contain the split that is present for the training of the swinunetr.
#create dataset.json and then after preprocessing / edit the splits

for fold in tqdm.tqdm(os.listdir(nnunet_raw)):
    if new_name2 in fold:
        fold_path = os.path.join(nnunet_raw,fold)
        #from the dataset.json get labels
        json_file = os.path.join(fold_path,'dataset_nnunetv1.json')
        with open(json_file,'r') as f:
            data = json.load(f)
        req_labels_in_order = {v:int(k) for k,v in data['labels'].items()}
        #print(req_labels_in_order)
        num_files = data['numTraining']
        #print(num_files)
        # train_file_path = os.path.join(fold_path,'imagesTr')
        # num_files = len([f for f in os.listdir(train_file_path) if f.endswith('.nii.gz')])
        # print(num_files)
        output_folder = fold_path
        channel_names = {"0":"CT"}
        labels = req_labels_in_order
        num_training_cases = num_files
        file_ending = '.nii.gz'
        dataset_name = fold.split('_')[1] + '_from_MSD'
        generate_dataset_json(output_folder=output_folder,
                              channel_names = channel_names,
                              labels = labels,
                              num_training_cases = num_training_cases,
                              file_ending = file_ending)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1082.08it/s]


In [37]:
#run to preprocess the files

In [38]:
#now create the custom splits.json file for each dataset.
nnunet_preprocessed = '/blue/kgong/s.kapoor/language_guided_segmentation/CLIP-Driven-Universal-Model_MSD_only/nnUNet_preprocessed'

In [41]:
#getting the dictionary of the requried folders
dic = {}
for fold in os.listdir(nnunet_preprocessed):
    fold_path = os.path.join(nnunet_preprocessed,fold)
    if os.path.isdir(fold_path) and fold[7:10] in ['003','006','007','008','009','010']:
        dic[fold]={'train':[],'val':[]}

In [42]:
dic

{'Dataset009_Spleen': {'train': [], 'val': []},
 'Dataset007_Pancreas': {'train': [], 'val': []},
 'Dataset008_HepaticVessel': {'train': [], 'val': []},
 'Dataset010_Colon': {'train': [], 'val': []},
 'Dataset006_Lung': {'train': [], 'val': []},
 'Dataset003_Liver': {'train': [], 'val': []}}

In [47]:
#create mappiong from Dataset to task
val_txt_file = './dataset/dataset_list/PAOT_10_inner_val.txt'
dic_map = {}
with open(train_txt_file,'r') as file:
    for line in file:

        line = line.strip()
        fold = line.split()[0].split('/')[1]
        if fold not in dic_map.keys():
            dic_map[fold] = fold.replace(old_name,new_name)

In [48]:
dic_map

{'Task03_Liver': 'Dataset003_Liver',
 'Task06_Lung': 'Dataset006_Lung',
 'Task07_Pancreas': 'Dataset007_Pancreas',
 'Task08_HepaticVessel': 'Dataset008_HepaticVessel',
 'Task09_Spleen': 'Dataset009_Spleen',
 'Task10_Colon': 'Dataset010_Colon'}

In [53]:
#getting the list of train files
train_txt_file = './dataset/dataset_list/PAOT_10_inner_train.txt'
with open(train_txt_file,'r') as file:

    for line in file:

        line = line.strip()
        gg = line.split()[0].split('/')
        fold = gg[1]
        act_fold = dic_map[fold]
        file = gg[-1].split('.')[0]
        dic[act_fold]['train'].append(file)

In [54]:
#getting the list of val files
val_txt_file = './dataset/dataset_list/PAOT_10_inner_val.txt'
with open(val_txt_file,'r') as file:

    for line in file:

        line = line.strip()
        gg = line.split()[0].split('/')
        fold = gg[1]
        act_fold = dic_map[fold]
        file = gg[-1].split('.')[0]
        dic[act_fold]['val'].append(file)

In [55]:
dic.keys()

dict_keys(['Dataset009_Spleen', 'Dataset007_Pancreas', 'Dataset008_HepaticVessel', 'Dataset010_Colon', 'Dataset006_Lung', 'Dataset003_Liver'])

In [56]:
dic['Dataset009_Spleen'].keys()

dict_keys(['train', 'val'])

In [57]:
len(dic['Dataset009_Spleen']['train'])

33

In [58]:
len(dic['Dataset009_Spleen']['val'])

8

In [61]:
for fold in tqdm.tqdm(os.listdir(nnunet_preprocessed)):
    if new_name2 in fold:
        fold_path = os.path.join(nnunet_preprocessed,fold)
        splits_path = os.path.join(fold_path,'splits_final.json')
        to_save = [dic[fold]]
        with open(splits_path,'w') as f:
            json.dump(to_save,f,indent=4)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1653.47it/s]


In [3]:
## to add modlaity indexes to the testing dataset of different CT organs
nnunet_raw = '/blue/kgong/s.kapoor/language_guided_segmentation/CLIP-Driven-Universal-Model_MSD_only/nnUNet_raw'

In [4]:
new_name2 = 'Dataset'

In [6]:
#now add _0000 to eachfile name cause modality.
des_fold = 'imagesTs'
for fold in tqdm.tqdm(os.listdir(nnunet_raw)):
    if new_name2 in fold:
        fold_path = os.path.join(nnunet_raw,fold)
        for subfold in os.listdir(fold_path):
            if not subfold.startswith('.') and des_fold in subfold:
                subfold_path = os.path.join(fold_path,subfold)
                for filename in os.listdir(subfold_path):
                    new_filename = filename[:-7]+'_0000.nii.gz'
                    filename_path = os.path.join(subfold_path,filename)
                    new_filename_path = os.path.join(subfold_path,new_filename)
                    #print(filename_path,new_filename_path)
                    os.rename(filename_path,new_filename_path)
                    #break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 20.43it/s]


In [7]:
import matplotlib.pyplot as plt